# What to scrape:

Event Data **This will be the hardest to scrape as I will need to specify the right game ids for the right league ids**:
1. Play by play data: Comments and Incidents ✅
2. Shotmaps ✅
3. Game Team Stats ❎ *Still needs some testing and data manipulation*
4. Game Team player lineups with stats ❎ *Needs extra parameters*

League Data **I need the league id AND their respective game year**:
1. Standings ❎ *Pretty easy to scrape so I didn't test*
2. Player stats ❎ *This is mostly done but I didn't test by year*



In [1]:
import json
import httpx
import pandas as pd
import asyncio
import numpy as np
from datetime import datetime
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import time

# Event Data

I want to attack the hard stuff first. 

Ideas:
* I'll need the game date and game ID, probably league ID in addition to help with merging
* Use asyncio for each league request so it's not overloading API but also still efficient
* Use threadpool executor to execute functions asynchronously and asyncio to execute requests asynchronously


## TODO: 
* Add in league info as a column
* Try to scrape multiple leagues
* Scrape the incidents, lineups, statistics

In [2]:
league_data = {
    'premier-league' : 17,
    'la-liga': 8,
    'serie-a': 23,
    'ligue-1': 34,
    'bundesliga': 35
    # Add more for later
}

# Establish the client
async def extract_json(url: str, object_: str, sleep:float=None):
    """ Extracts json. Will be using this multiple times so it's up here. """
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, timeout = None)
            response.raise_for_status()
            if sleep:
                await asyncio.sleep(sleep)
            return response.json().get(object_)
        except httpx.HTTPError:
            print(f'url {url} not working')
            return None

async def fetch_events(dates:list[str], object_:str, ids:list[int]):
    """ Finds all game ids based on a list of date parameters. """
    game_ids = []
    
    async def fetch_dates(date:str):
        url = f'https://www.sofascore.com/api/v1/sport/football/scheduled-events/{date}'
        json = await extract_json(url, object_)
        return [event.get('id') for event in json if event.get('tournament', {}).get('uniqueTournament', {}).get('id') in (ids)]
    
    tasks = [fetch_dates(date) for date in dates]
    for result in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc = 'Processing Dates'):
        # Some tasks take longer which is why dates may be off
        if result is not None:
            game_ids.extend(await result)
    
    return game_ids
    
    
async def filter_unique_ids(game_ids: list[int]):
    """ This takes all the game ids, finds the event and store the game id, date, and league name """
    unique_games = {}
    
    game_ids = set(game_ids)
    
    loop = asyncio.get_event_loop()
    
    def fetch_event(game_id, loop):
        url = f'https://www.sofascore.com/api/v1/event/{game_id}'
        event = asyncio.run(extract_json(url, 'event'))
        game_date = datetime.fromtimestamp(event['startTimestamp']).date()
        league_name = event['tournament']['name']
        return game_id, game_date, league_name
    
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, fetch_event, game_id, loop) for game_id in game_ids]
        
        for future in tqdm(asyncio.as_completed(tasks), total=len(game_ids), desc='Filtering Game IDs'):
            result = await future
            if result:
                game_id, game_date, league_name = result
                if game_id not in unique_games or unique_games[game_id][0] < game_date:
                    unique_games[game_id] = (game_date, league_name)
    
    return unique_games

async def fetch_data(game_id:int, object_:str):
    """ Loads the data based on whatever parameter: comments, incidents and shotmap. """
    url = f'https://www.sofascore.com/api/v1/event/{game_id}/{object_}' # To add params comments, incidents, and shotmap. Lineups needs another parameter home or away
    json = await extract_json(url, object_, sleep=0.5) # can adjust this to 1 if needed
    return json

start_date = "2023-08-10"
end_date = "2024-05-24"
dates = pd.date_range(start=start_date, end=end_date).strftime("%Y-%m-%d").tolist()

events = await fetch_events(dates, 'events', league_data.values())
unique_game_ids = await filter_unique_ids(events)

async def process_game(game_id: int, game_date, league_name):
    """ Adds the game id, date, and league name into the loaded json data. """
    data = []
    comments = await fetch_data(game_id, 'comments')
    if comments:
        for comment in comments:
            combined = comment.copy()
            combined.update({'game_id': game_id, 'date': game_date, 'league': league_name})
            data.append(combined)
    return data


data = []

tasks = [process_game(game_id, game_date, league_name) for game_id, (game_date, league_name) in unique_game_ids.items()]
for result in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc='Processing Game IDs'):
    comments = await result
    data.extend(comments)



Processing Game IDs:   0%|                   | 2/1750 [00:24<4:57:10, 10.20s/it]

url https://www.sofascore.com/api/v1/event/11896302/comments not working
url https://www.sofascore.com/api/v1/event/11391350/comments not working
url https://www.sofascore.com/api/v1/event/11396405/comments not working
url https://www.sofascore.com/api/v1/event/11392858/comments not working
url https://www.sofascore.com/api/v1/event/11391359/comments not working
url https://www.sofascore.com/api/v1/event/11391349/comments not working
url https://www.sofascore.com/api/v1/event/11352311/comments not working
url https://www.sofascore.com/api/v1/event/11392896/comments not working
url https://www.sofascore.com/api/v1/event/11391351/comments not working
url https://www.sofascore.com/api/v1/event/11407418/comments not working
url https://www.sofascore.com/api/v1/event/11368603/comments not working
url https://www.sofascore.com/api/v1/event/11407482/comments not working
url https://www.sofascore.com/api/v1/event/11391102/comments not working
url https://www.sofascore.com/api/v1/event/11407413

Processing Game IDs:   2%|▍                   | 41/1750 [00:24<07:23,  3.86it/s]

url https://www.sofascore.com/api/v1/event/11391353/comments not working
url https://www.sofascore.com/api/v1/event/11396474/comments not working
url https://www.sofascore.com/api/v1/event/11391346/comments not working
url https://www.sofascore.com/api/v1/event/11391358/comments not working
url https://www.sofascore.com/api/v1/event/11391357/comments not working
url https://www.sofascore.com/api/v1/event/11392852/comments not working
url https://www.sofascore.com/api/v1/event/11407490/comments not working
url https://www.sofascore.com/api/v1/event/11396444/comments not working
url https://www.sofascore.com/api/v1/event/11392867/comments not working
url https://www.sofascore.com/api/v1/event/11407481/comments not working
url https://www.sofascore.com/api/v1/event/11396427/comments not working
url https://www.sofascore.com/api/v1/event/11391356/comments not working
url https://www.sofascore.com/api/v1/event/11407466/comments not working
url https://www.sofascore.com/api/v1/event/11396460

Processing Game IDs:   3%|▋                   | 61/1750 [00:25<04:01,  7.00it/s]

...

Processing Game IDs:  98%|█████████████████▋| 1723/1750 [00:34<00:00, 67.50it/s]

url https://www.sofascore.com/api/v1/event/11369443/comments not working


Processing Game IDs:  99%|█████████████████▉| 1738/1750 [00:35<00:00, 50.34it/s]

url https://www.sofascore.com/api/v1/event/11396365/comments not working
url https://www.sofascore.com/api/v1/event/11352389/comments not working
url https://www.sofascore.com/api/v1/event/11368746/comments not working
url https://www.sofascore.com/api/v1/event/11368750/comments not working
url https://www.sofascore.com/api/v1/event/11406775/comments not working
url https://www.sofascore.com/api/v1/event/11352376/comments not working
url https://www.sofascore.com/api/v1/event/11369378/comments not working
url https://www.sofascore.com/api/v1/event/11352345/comments not working
url https://www.sofascore.com/api/v1/event/11392810/comments not working
url https://www.sofascore.com/api/v1/event/11391333/comments not working
url https://www.sofascore.com/api/v1/event/11352424/comments not working


Processing Game IDs: 100%|██████████████████| 1750/1750 [00:36<00:00, 47.30it/s]


In [3]:
df = pd.json_normalize(data)

In [4]:
import janitor

df = janitor.clean_names(df)
df['date'] = df['date'].astype(str)

#df = df.loc[:, (df.count() >= 100)]

In [5]:
df

,text,type,id,time,game_id,date,league,periodname,ishome,player_name,...,assist1_slug,assist1_shortname,assist1_position,assist1_jerseynumber,assist1_usercount,assist1_id,assist1_fieldtranslations_nametranslation_ar,assist1_fieldtranslations_shortnametranslation_ar,playerin_firstname,playerin_lastname
0,"Match ends, Lyon 3, Monaco 2.",matchEnded,23703576,0,11391263,2024-04-28,Ligue 1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Second Half ends, Lyon 3, Monaco 2.",endSecondHalf,23703519,95,11391263,2024-04-28,Ligue 1,2ND,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Foul by Maxence Caqueret (Lyon).,freeKickLost,23703500,94,11391263,2024-04-28,Ligue 1,2ND,True,Maxence Caqueret,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wissam Ben Yedder (Monaco) wins a free kick in...,freeKickWon,23703499,94,11391263,2024-04-28,Ligue 1,2ND,False,Wissam Ben Yedder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Substitution, Lyon. Mama Baldé replaces Alexan...",substitution,23703463,91,11391263,2024-04-28,Ligue 1,2ND,True,Mama Baldé,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30719,Delay over. They are ready to continue.,endDelay,23681017,6,11391300,2024-04-27,Ligue 1,1ST,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30720,Delay in match (Paris Saint Germain).,startDelay,23681018,6,11391300,2024-04-27,Ligue 1,1ST,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30721,"Offside, Paris Saint Germain. Achraf Hakimi is...",offside,23680914,2,11391300,2024-04-27,Ligue 1,1ST,True,Ousmane Dembélé,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30722,Attempt blocked. Ousmane Dembélé (Paris Saint ...,shotBlocked,23680880,2,11391300,2024-04-27,Ligue 1,1ST,True,Ousmane Dembélé,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
